<a href="https://colab.research.google.com/github/Tixonmavrin/Zindi-Zimnat-Insurance-Recommendation-Challenge/blob/master/Baseline2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

     |████████████████████████████████| 65.9MB 59kB/s 


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

In [ ]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
submission = pd.read_csv('SampleSubmission.csv')

In [ ]:
print(train.shape)
train.head()

(29132, 29)


,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [ ]:
print(test.shape)
test.head()

(10000, 29)


,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,1/12/2018,M,M,1984,94KC,DZRV,90QI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H6141K3,1/10/2019,M,M,1996,1X1H,J9SY,90QI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,1/1/2020,F,W,1968,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,1/2/2019,M,M,1989,94KC,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,1/2/2020,F,M,1982,UAOD,0KID,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
print(submission.shape)
submission.head()

(210000, 2)


,ID X PCODE,Label
0,F86J5PC X P5DA,0
1,F86J5PC X RIBP,0
2,F86J5PC X 8NN1,0
3,F86J5PC X 7POT,0
4,F86J5PC X 66FJ,0


In [ ]:
np_data = []
train_columns = train.columns
for v in tqdm(train.values):
  info = v[:8]
  binary = v[8:]
  index_n = [k for k, i in enumerate(binary) if i == 1]
  for i in index_n:
    for k in range(len(binary)):
      if (k not in index_n) or (k == i):
        binary_0 = list(copy.copy(binary))
        binary_0[i] = 0
        if k == i:
          np_data.append(list(info) + binary_0 + [train_columns[8+k]] + [1])
        else:
          np_data.append(list(info) + binary_0 + [train_columns[8+k]] + [0])

df_data = pd.DataFrame(np_data)
df_data.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'target']

100%|██████████| 29132/29132 [00:09<00:00, 3214.86it/s]


In [ ]:
np_data_test = []
answ_test = []
test_columns = test.columns
for v in tqdm(test.values):
  info = v[:8]
  binary = v[8:]
  index_n = [k for k, i in enumerate(binary) if i == 1]
  for k in range(len(binary)):
    if k not in index_n:
      np_data_test.append(list(info) + list(binary) + [test_columns[8+k]])

df_data_test = pd.DataFrame(np_data_test)
df_data_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred']

100%|██████████| 10000/10000 [00:01<00:00, 8131.91it/s]


In [ ]:
df_data['date1'] = df_data['join_date'].apply(lambda x: x.split('/')[0] if (x == x) else np.nan)
df_data['date2'] = df_data['join_date'].apply(lambda x: x.split('/')[1] if (x == x) else np.nan)
df_data['date3'] = df_data['join_date'].apply(lambda x: x.split('/')[2] if (x == x) else np.nan)
df_data.drop('join_date', axis=1, inplace=True)

In [ ]:
df_data_test['date1'] = df_data_test['join_date'].apply(lambda x: x.split('/')[0] if (x == x) else np.nan)
df_data_test['date2'] = df_data_test['join_date'].apply(lambda x: x.split('/')[1] if (x == x) else np.nan)
df_data_test['date3'] = df_data_test['join_date'].apply(lambda x: x.split('/')[2] if (x == x) else np.nan)
df_data_test.drop('join_date', axis=1, inplace=True)

In [ ]:
from catboost import CatBoostClassifier
cat_features = ['sex', 'marital_status', 'branch_code',
       'occupation_code', 'occupation_category_code', 'product_pred']

model = CatBoostClassifier()
model.fit(df_data.drop(['ID', 'target'], axis=1), df_data['target'], cat_features)

Learning rate set to 0.219943
0:	learn: 0.2674929	total: 1.89s	remaining: 31m 23s
1:	learn: 0.1461136	total: 3.46s	remaining: 28m 48s
2:	learn: 0.1037973	total: 5.18s	remaining: 28m 42s
3:	learn: 0.0817524	total: 6.44s	remaining: 26m 43s
4:	learn: 0.0735401	total: 7.73s	remaining: 25m 38s
5:	learn: 0.0666916	total: 9.31s	remaining: 25m 43s
6:	learn: 0.0611330	total: 10.8s	remaining: 25m 27s
7:	learn: 0.0584565	total: 12.1s	remaining: 25m
8:	learn: 0.0537983	total: 13.7s	remaining: 25m 8s
9:	learn: 0.0526617	total: 14.9s	remaining: 24m 36s
10:	learn: 0.0514694	total: 16.5s	remaining: 24m 44s
11:	learn: 0.0506059	total: 18.3s	remaining: 25m 3s
12:	learn: 0.0491453	total: 19.9s	remaining: 25m 12s
13:	learn: 0.0483386	total: 21.5s	remaining: 25m 16s
14:	learn: 0.0479079	total: 22.8s	remaining: 24m 55s
15:	learn: 0.0473802	total: 24.4s	remaining: 24m 57s
16:	learn: 0.0470237	total: 26s	remaining: 25m 1s
17:	learn: 0.0468184	total: 27.8s	remaining: 25m 14s
18:	learn: 0.0463269	total: 29.5s	r

In [ ]:
preds_proba = model.predict_proba(df_data_test.drop(['ID',], axis=1))
df_answer = df_data_test[['ID', 'product_pred']]
df_answer['target'] = preds_proba[:,1]
df_answer['ID X PCODE'] = df_answer['ID'] + ' X ' + df_answer['product_pred']
df_answer.drop(['ID', 'product_pred'], axis=1, inplace=True)
df_answer.rename(columns={'target':'Label'}, inplace=True)
df_answer = submission[submission['ID X PCODE'].isin(list(set(list(submission['ID X PCODE'])) - set(list(df_answer['ID X PCODE']))))].append(df_answer)
df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('submission2.csv', index=False)